In [3]:
import folium
import os
import json
import pandas as pd
import branca
#from shapely.geometry import MultiPolygon, shape, mapping

#change dir to the location of the data_bourgogne.json file which contains only locations of Burgundy wine
os.chdir('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/jsons')
%run '/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/scripts/function_addlayer.py'

In [4]:
# Open the file
with open('data_bourgogne_wtype.json', encoding = 'utf-8') as json_file:
    data = json.load(json_file)
    
file = open("/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/scripts/grandscrus.txt",'r')
grandscrus = file.read().split('\n')

for key in data.keys():
    for gc in grandscrus:
        if data[key]['properties']['appellation'] == gc :
            data[key]['properties']['Grand Cru'] = 1

In [5]:
df = pd.read_csv('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/scripts/cote_beaune.csv', header = None)
df.columns = ['appelation', 'type vin']

In [6]:
appelation = [x.lower() for x in df['appelation']]

In [7]:
data_cote2beaune = {}
data_rouge = {}
data_blanc = {}

for key in data.keys():
    if data[key]['properties']['appellation'].lower() in appelation :
        data_cote2beaune[key] = data[key]
        
for key in data_cote2beaune.keys():
    if data_cote2beaune[key]['properties']['type_vin'] == 'rouge':
        if 'mousseux' not in data_cote2beaune[key]['properties']['appellation'].lower() and "Crémant" not in data_cote2beaune[key]['properties']['appellation'] :
            data_rouge[key] = data_cote2beaune[key]
        
    if data_cote2beaune[key]['properties']['type_vin'] == 'blanc':
        if 'mousseux' not in data_cote2beaune[key]['properties']['appellation'].lower() and "Crémant" not in data_cote2beaune[key]['properties']['appellation']:
            data_blanc[key] = data_cote2beaune[key]

In [8]:
style_redwine = {'fillColor': '#DC1A40', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}
style_whitewine = {'fillColor': '#EAE010', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}

os.chdir('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE')

wine_map = folium.Map(location=[47.0333, 4.83333],
                      height=500,
                      tiles = "Stamen Terrain",
                      zoom_start=12,
                      prefer_canvas=True)

# If no image is wanted, call add_layertomap_no_image instead of add_layertomap.
# works exactly as previously.
add_layertomap_no_image(data_rouge, style_redwine, 'Côte de Beaune Rouge', wine_map)
add_layertomap_no_image(data_blanc, style_whitewine, 'Côte de Beaune Blanc', wine_map, show = False)
folium.LayerControl().add_to(wine_map)

#wine_map.save(os.path.join('results', 'cote2beaune.html'))
wine_map.save('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/cote2beaune.html')
#display(wine_map)